<a href="https://colab.research.google.com/github/mark-ford123/applied_ml_cw_2025_mf/blob/main/coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d nolasthitnotomorrow/radioml2016-deepsigcom
!unzip "/content/radioml2016-deepsigcom.zip" -d "/content/"

Dataset URL: https://www.kaggle.com/datasets/nolasthitnotomorrow/radioml2016-deepsigcom
License(s): CC-BY-NC-SA-4.0
 99% 276M/279M [00:12<00:00, 27.7MB/s]
100% 279M/279M [00:12<00:00, 23.3MB/s]
Archive:  /content/radioml2016-deepsigcom.zip
  inflating: /content/RML2016.10a_dict.pkl  


In [6]:
import pickle
with open("RML2016.10a_dict.pkl", "rb") as f:
  Xd = pickle._Unpickler(f)
  Xd.encoding = "latin1"
  Xd = Xd.load()